In [1]:
%cd ../../../../

/home/users/dmoreno2016


In [2]:
import toml
import os

from src.models.astromer_1 import get_ASTROMER

%load_ext autoreload
%autoreload 2

2023-09-18 01:24:28.561955: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
pt_folder = 'presentation/experiments/astromer_1_pe/results/pretraining/probed_0.2/macho_pe_nontrainable-2023-09-18_01-12-37'

with open(os.path.join(pt_folder, 'config.toml'), 'r') as f:
    model_config = toml.load(f)

with open(os.path.join(pt_folder, 'pe_config.toml'), 'r') as f:
    pe_config = toml.load(f)

astromer = get_ASTROMER(num_layers=model_config['num_layers'],
                        num_heads=model_config['num_heads'],
                        head_dim=model_config['head_dim'],
                        mixer_size=model_config['mixer'],
                        dropout=model_config['dropout'],
                        pe_type=model_config['pe_type'],
                        pe_config=pe_config,
                        window_size=model_config['window_size'],
                        encoder_mode=model_config['encoder_mode'],
                        average_layers=model_config['avg_layers'])

2023-09-18 01:24:30.628694: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-18 01:24:32.253827: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 45394 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:c1:00.0, compute capability: 8.6
2023-09-18 01:24:32.254751: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 156 MB memory:  -> device: 1, name: NVIDIA RTX A6000, pci bus id: 0000:c2:00.0, compute capability: 8.6


In [4]:
astromer.summary()

Model: "ASTROMER-1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 mask (InputLayer)              [(None, 200, 1)]     0           []                               
                                                                                                  
 input (InputLayer)             [(None, 200, 1)]     0           []                               
                                                                                                  
 times (InputLayer)             [(None, 200, 1)]     0           []                               
                                                                                                  
 encoder (Encoder)              (None, 200, 256)     660992      ['mask[0][0]',                   
                                                                  'input[0][0]',         

In [5]:
for layer in astromer.layers:
    print(f'Name layers: {layer.name}')
    print(f'Number of parameters: {layer.count_params()}')

Name layers: mask
Number of parameters: 0
Name layers: input
Number of parameters: 0
Name layers: times
Number of parameters: 0
Name layers: encoder
Number of parameters: 660992
Name layers: regression
Number of parameters: 769


In [20]:
import datetime

datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

'2023-09-18_01-12-37'

In [16]:
pe_list = ['positional_embedding', 'pos_time_modulation']
pe_weights_names = ['alpha_sin']

for layer in astromer.get_layer('encoder').layers:
    print(f'Name layers: {layer.name}')

    if layer.name.find('att_') != -1:
        print(f'Number of parameters: {layer.count_params()}')

Name layers: inp_transform
Name layers: pos_time_modulation
Name layers: att_layer_0
Number of parameters: 330112
Name layers: att_layer_1
Number of parameters: 330112
Name layers: dropout_4


In [14]:
astromer.summary()

Model: "ASTROMER-1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 mask (InputLayer)              [(None, 200, 1)]     0           []                               
                                                                                                  
 input (InputLayer)             [(None, 200, 1)]     0           []                               
                                                                                                  
 times (InputLayer)             [(None, 200, 1)]     0           []                               
                                                                                                  
 encoder (Encoder)              (None, 200, 256)     726272      ['mask[0][0]',                   
                                                                  'input[0][0]',         

In [8]:
path_pe_config = 'presentation/experiments/astromer_1_pe/pe_config.toml'
with open(path_pe_config, mode="r") as fp:
    pe_config = toml.load(fp)

In [13]:
pe_config

{'general': {'pe_list': ['positional_embedding',
   'pe_mlp',
   'pe_rnn',
   'pos_time_modulation']},
 'pe': {'base': 1000,
  'c': 1,
  'use_mjd': True,
  'pe_trainable': False,
  'initializer': 'pe_astromer',
  'min_period': 0.01,
  'max_period': 1825},
 'pe_mlp': {'m_layers': 128, 'use_mjd': True},
 'pe_rnn': {'rnn_type': 'gru', 'use_mjd': True},
 'pe_tm': {'T_max': 1500,
  'H': 64,
  'use_mjd': True,
  'pe_trainable': True,
  'initializer': 'random_normal'}}

In [14]:
dict_pe = {
    'general': pe_config['general'],
    'pe': pe_config['pe']
    }
dict_pe

{'general': {'pe_list': ['positional_embedding',
   'pe_mlp',
   'pe_rnn',
   'pos_time_modulation']},
 'pe': {'base': 1000,
  'c': 1,
  'use_mjd': True,
  'pe_trainable': False,
  'initializer': 'pe_astromer',
  'min_period': 0.01,
  'max_period': 1825}}